## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


21525 строк всего и только 19351 полных в двух столбцах из 12
это примерно 10%

In [1]:
df.head()

NameError: name 'df' is not defined

Уже на первый взгляд становится ясно что столбцы 'days_employed', 'education', 'total_income',  'purpose' потребуют обработки. 

Теперь узнаем точное количество пропусков

In [3]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

2174 пропуска это 9,9% общего количества.

Что у нас с дубликатами?

In [4]:
df.duplicated().sum()

54

Теперь будем посмотреть остальные странности в колонках

In [5]:
df.groupby('children')['children'].value_counts()

children  children
-1        -1             47
 0         0          14149
 1         1           4818
 2         2           2055
 3         3            330
 4         4             41
 5         5              9
 20        20            76
Name: children, dtype: int64

Не совсем понятно что значит -1 и 20. А в остальном конечно большинство бездетные. Если мы избавимся от 123 строк со странными значениями о количестве детей показатели нам это не испортит ведь это всего примерно 0,005% от общего количетсва строк.

Теперь посмотрим общее число строк трудового стажа с отрицательными значениями

In [6]:
df[df['days_employed'] < 0]['days_employed'].count()

15906

Теперь положительные...

In [7]:
df[df['days_employed'] > 0]['days_employed'].count()

3445

А если так?

In [8]:
df[df['days_employed'] > 300000]['days_employed']

4        340266.072047
18       400281.136913
24       338551.952911
25       363548.489348
30       335581.668515
             ...      
21505    338904.866406
21508    386497.714078
21509    362161.054124
21518    373995.710838
21521    343937.404131
Name: days_employed, Length: 3445, dtype: float64

Это же в днях верно? =) 100% из 100% людей с положительным стажем работы в нашей таблице могут сказать: "Столько не живут сколько я работал/а"

Можно подумать когда мы посмотрим количество людей по возрасту примерно 3500 человек будет возраста 900+ лет =)

In [9]:
df.sort_values('dob_years')['dob_years'].value_counts().sort_index()


0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Итак, долгожители не наблюдаются. Казалось бы 101 человек неопределённого возраста так-же кандидаты на выбывание из таблицы, только избавление от них никак не повлияет на информацию ответов в следующих вопросах...

    1)Есть ли зависимость между наличием детей и возвратом кредита в срок?
    2)Есть ли зависимость между семейным положением и возвратом кредита в срок?
    3)Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    4)Как разные цели кредита влияют на его возврат в срок?
    
Данные об образовании

In [10]:
df.sort_values('education')['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Тут если всё привести в единый вид, то вроде всё нормально

Теперь семейное положение

In [11]:
df['family_status'].value_counts()


женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Гендер

In [12]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Клиентов женщин в два раза больше чем мужчин, а вот последняя строка к нам пришла из Европы =)

Занятость

In [13]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Т.к. доходы у всех разные, будем посмотреть кто у нас есть с уровнем дохода 0 и пропуски

In [14]:
df[df['total_income'] == 0]['total_income'].count()

0

In [15]:
df[df['total_income'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Каковы же цели получений кредитов?

In [16]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Выделяются одни и те же цели в разных их интерпретациях

### Вывод

1)Таблица состоит из 21525 строк. Из них 54 дубликата количество которых может измениться после предобработки данных.

2)Одинаковое количество пропусков в столбцах 'days_employed' и 'total_income' - 2174. В каждом строке где отсутствует стаж, отсутствует и доход. Можно подумать что есть какая-то закономерность, кроме этого, например это встречается только у пенсионеров,однако это не так. Похоже что отсутствие значений случайны.

3)Т.к столбец 'days_employed' содержит слово дней будем приводить дробные значения к целыми и отрицательные значения - делать положительными. Аномально высокий стаж приведём только к целым значениям. Делать с ними что то ещё не представляется необходимым, ведь это не влияет на статистику в наших ответах

4)Столбец 'total_income' необходим для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" по этому от пустых значениц в них нет пользы. Попробуем их восстановить медианными значениями по двум категориям: образование и тип занятости.

5)В столбце дети количество -1 и 20 выбиваются из общей картины, от них лучше избавится т.к. общей информации это не испортит.

6)В столбце 'education' есть категория с разным регистром, здесь поможет метод lowercase.

7)101 нулевое значение в столбце 'dob_years' на вопросы банка не влияет, делать ничего не будем.

8)В 'family_status' только категория Не женат / не замужем выбивается из общей картины. lowercase=)

9)Человека неопределившегося со своим полом в графе 'gender' мы трогать не будем.

10) В столбце цели кредита избавиться от разных формулировок одного и того же нам поможет лемматизация


### Шаг 2. Предобработка данных

### Обработка пропусков

Перед составлением сводной таблицы для заполнений пропусков медианным значением в столбцх 'days_employed' и 'total_income' приведем все значения столбца 'education' к единому регистру

In [17]:
df['education'] = df['education'].str.lower()

Теперь собственно составление и вывод самой таблицы

In [18]:
df_median = pd.pivot_table(df, index=['education', 'income_type'], values=['days_employed', 'total_income'],
    aggfunc='median')

df_median

days_employed   total_income
education           income_type                                  
высшее              безработный      395302.838654  202722.511368
                    госслужащий       -2531.034209  172511.107016
                    компаньон         -1454.659104  201785.400018
                    пенсионер        366158.526428  144240.768611
                    предприниматель    -520.848083  499163.144947
                    сотрудник         -1556.457266  165640.744634
                    студент            -578.751554   98201.625314
начальное           госслужащий       -2787.767403  148339.290825
                    компаньон         -1151.634460  136798.905143
                    пенсионер        360264.985350  102598.653164
                    сотрудник         -1197.176853  125994.910603
неоконченное высшее госслужащий       -1885.183639  160592.345303
                    компаньон         -1017.888120  179867.152890
                    пенсионер        372250.501660  120136.896353
                    сотрудник         -1197.676879  151308.937846
среднее             безработный      337524.466835   59956.991984
                    в декрете         -3296.759962   53829.130729
                    госслужащий       -2857.770974  136652.970357
                    компаньон         -1670.809068  159070.690289
                    пенсионер        365025.338867  114842.854099
                    сотрудник         -1613.581420  136555.108821
ученая степень      госслужащий       -5968.075884  111392.231107
                    пенсионер        356930.517546  177088.845999
                    сотрудник         -2351.431934  198570.757322

Теперь напишем и применим функцию для заполнения пропуска дней стажа на основе сводной таблицы


In [19]:
def days_median_back(row):
    return df_median.loc[row['education']].loc[row['income_type']]['days_employed']

df.loc[df['days_employed'].isnull(), 'days_employed'] = df.apply(days_median_back, axis=1)

Функция и её приминение для заполнения пропусков уровня дохода на основе сводной таблицы

In [20]:
def incom_median_back(row):
    return df_median.loc[row['education']].loc[row['income_type']]['total_income']

df.loc[df['total_income'].isnull(), 'total_income'] = df.apply(incom_median_back, axis=1)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропусков в стобцах с которыми мы работали не наблюдается


Теперь избавимся от лишних детишек =) Шутка

In [22]:
df = df[(df['children'] >= 0) & (df['children'] <= 5)]
df.sort_values('children')['children'].value_counts().sort_index()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       21402 non-null float64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null float64
purpose             21402 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

Мы избавились от пропусков. Остались только данные для анализа.


### Замена типа данных

In [24]:
df['days_employed'] = df['days_employed'].apply(abs)
df['days_employed'] = df['days_employed'].astype('int')
df[df['days_employed'] < 0]['days_employed'].count()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0

In [25]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Числовой функцией abs() метода apply() преобразовали количество дней стажа в целочисленный тип дополнительно изменив отрицательные значения на положительные

### Обработка дубликатов

In [26]:
df['family_status'].replace(['Не женат / не замужем'], "не женат / не замужем", inplace=False)
df['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

### Вывод

После приведения столбца 'education' к единому реестру, в Обработки пропусков необходимость применять метод drop_duplicates() не нужно. Их 0

In [1]:
df.duplicated().sum()

NameError: name 'df' is not defined

### Лемматизация

Здесь будем исмользовать подсчет различных значений в списке и Mystem.

In [27]:
from collections import Counter
from pymystem3 import Mystem
m = Mystem()
purposes = []
for i in df['purpose']:
    lemma = m.lemmatize(i)
    purposes += filter(lambda l: l not in [' ', '\n', 'с', 'со', 'на'], lemma)
print(Counter(purposes))

Counter({'недвижимость': 6330, 'покупка': 5880, 'жилье': 4450, 'автомобиль': 4288, 'образование': 3997, 'операция': 2593, 'свадьба': 2337, 'свой': 2224, 'строительство': 1870, 'высокий': 1368, 'получение': 1311, 'коммерческий': 1307, 'для': 1291, 'жилой': 1225, 'сделка': 939, 'заниматься': 908, 'дополнительный': 902, 'проведение': 772, 'сыграть': 769, 'сдача': 651, 'семья': 640, 'собственный': 628, 'ремонт': 609, 'подержанный': 484, 'подержать': 472, 'приобретение': 460, 'профильный': 432})


Как известно покупка может быть недвижимости, автомобиля и т.п. По этому если взять наиболее часто повторяемые категории то останутся...

In [28]:
five_category = ['свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование']

Т.к. почитав рекомендации наставников в чате project мы знаем что лучше добавить новый столбец то напишем функцию для этого

In [29]:
def purpose_category(text):
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        for category in five_category:
            if category in lemma:
                return category
            
# Попробуем на деле
df['purpose_category'] = df['purpose'].apply(purpose_category)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [30]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,жилье


### Вывод

Сведя множество интерпретаций целей кредита, получили 5 категорий и дополнительный столбец в таблице по которому лече ориентироваться.

### Категоризация данных

Что бы ответить на заданные вопросы нам предстоит категоризировать клиентов банка по налицию детей, по семейному положению, по уровню дохода и по целям кредита, категоризация которых была сделана во время лемматизации.

Наличию детей в семье будут даны 4 категории

1 - бездетная

2 - один ребёнок

3 - два ребёнка

4 - многодетная

In [31]:
def category_children(count):
    categories = {
        0: 'бездетная',
        1: 'однодетная',
        2: 'малодетная'
    }

    return categories.get(count, 'многодетная')

In [32]:
df['children_category'] = df['children'].apply(category_children)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,жилье,однодетная
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,однодетная
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,жилье,бездетная
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,многодетная
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,бездетная
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,жилье,бездетная
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,жилье,бездетная
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование,бездетная
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,малодетная
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,жилье,бездетная


Категоризируя клиентов по семейному предположению, видно что у банка данные подразделяются в столбце 'family_status_id' на 2 типа. Кто то есть и никого нет) Возьмем за основу этот столбец и сделаем 2 категории

В столбце 'total_income' для начала все значения приведем к целому, после чего найдем минимальное значение, медиану и максимальное, для образования категорий.

In [34]:
df['total_income'] = df['total_income'].astype('int')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
def category_family(value):
    if value == 'женат / замужем':
        return 'в браке'
    
    return 'не в браке'

In [62]:
df['family_status_category'] = df['family_status'].apply(category_family)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,income_category,family_status_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,однодетная,выше среднего,в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,однодетная,средний,в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,бездетная,выше среднего,в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетная,выше среднего,в браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,бездетная,выше среднего,не в браке
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье,бездетная,выше среднего,не в браке
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье,бездетная,выше среднего,в браке
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,бездетная,средний,в браке
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,малодетная,средний,не в браке
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,бездетная,выше среднего,в браке


In [64]:
incom_min = df['total_income'].min().astype('int')
incom_min

20667

In [65]:
incom_median = df['total_income'].median()
incom_median

143494.5

In [66]:
incom_max = df['total_income'].max()
incom_max

2265604

Создание функции распределяющей на категории дохода

In [67]:
def categorize_income(value):
    if value < 50000:
        return 'низкий'
    elif value <= incom_median:
        return 'средний'
    elif value < 350000:
        return 'выше среднего'
    else:
        return 'высокий'

In [68]:
df['income_category'] = df['total_income'].apply(categorize_income)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [69]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,income_category,family_status_category
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,однодетная,выше среднего,не в браке
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,бездетная,выше среднего,в браке
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,однодетная,средний,не в браке
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,многодетная,выше среднего,в браке
21524,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,автомобиль,малодетная,средний,в браке


Цели кредита мы уже категоризировали на этапе лемматизации

1) свадьба

2) недвижимость

3) жилье

4) автомобиль

5) образование


### Вывод

Данные категоризированны, настало время их группировки для ответов на вопросы бизнеса

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [70]:
pd.pivot_table(df, index='children_category', values='debt')

,debt
children_category,
бездетная,0.075129
малодетная,0.094404
многодетная,0.081579
однодетная,0.092154


### Вывод

Прямой зависимости между наличием детей и возвратом кредита в срок нет:

Бездетные семьи чаще выплачивают кредит 7.5%.

9.2% однодетных клиентов имели задолжности по кредитам

С двумя имеют показателем 9.4%.

Многодетные семьи реже в должниках в 8.1% случаев



- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [71]:
pd.pivot_table(df, index='family_status_category', values='debt')

,debt
family_status_category,
в браке,0.075354
не в браке,0.088462


### Вывод

Клиенты, состоящие в браке, на 1.3% реже остаются в должниках

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [72]:
pd.pivot_table(df, index='income_category', values='debt')

,debt
income_category,
высокий,0.064177
выше среднего,0.078220
низкий,0.061995
средний,0.085576


### Вывод

Возврат кредита зависит от уровня дохода клиента: чем выше доход, тем меньше % задолжностей. Однако
люди с низким доходом имеют самый низкий процент задолжностей по кредитам: 6.2%, а вот люди со средним – самый высокий 8.5%.

- Как разные цели кредита влияют на его возврат в срок?

In [73]:
pd.pivot_table(df, index='purpose_category', values='debt')

,debt
purpose_category,
автомобиль,0.093284
жилье,0.069213
недвижимость,0.074566
образование,0.092319
свадьба,0.078306


### Вывод

Хуже всех с задолжностями по кредитам у клиентов занимавших на автомобили и образование, средний показатель у людей бравших на свадьбу.
Лучше всех обстоят дела у заемщиков на недвижимость и свадьбу

### Шаг 4. Общий вывод

Подытожив выводы сделанные выше, можно сказать что максимальный процент должников по любым категориям не превышает 10. Лучшими клиентами кредитного отдела банка после нашего исследования становятся Бездетные, состоящие в браке заёмщики с высоким уровнем дохода, берущие кредит на жилье =)